# RubyPyMill Example: Ruby vs Python Radar Chart

This notebook is designed to be executed by **RubyPyMill / Papermill**.

- It defines parameters in the `parameters` cell.
- It generates a demo CSV from an in-notebook table (`generate_data`).
- It renders a radar chart to a PNG file (`graph_output`).
- Optionally displays the image (`graph_view`).

## Tag layout (for RubyPyMill)

- `parameters`
- `preprocess`
- `generate_data`
- `graph_output`
- `graph_view`


# parameters

この Notebook の実行パラメータ。未指定でも動作する。この Notebook の実行パラメータ。未指定でも動作する。

In [ ]:
# parameters

csv_path = "examples/data/lang_skills.csv"
out_png  = "examples/outputs/lang_radar.png"

title = "Ruby vs Python: Strength Profile (1–5)"
max_score = 5
show_plot = False
fill_alpha = 0.10

lang_order = "Ruby,Python"
dimension_order = ""  # empty => use CSV appearance order

run_id = "demo"


# preprocess

環境設定、関数設定

In [ ]:
# preprocess

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ensure output directory exists
os.makedirs(os.path.dirname(out_png), exist_ok=True)

def split_or_none(s: str):
    """Split comma-separated string into list, or return None when empty."""
    return [x.strip() for x in s.split(",")] if s else None


# generate_data

Ruby と Python の得意分野を 1–5 で定義する。

このセルでは、Ruby と Python の「得意分野の傾向」を  
1〜5 の相対スコアとして定義しています。

ここでの数値は、言語の優劣を示すものではなく、  
**PoC や業務自動化における役割分担のイメージ**を表したものです。

- Ruby は DSL・制御・オーケストレーション寄り
- Python は データ処理・可視化・Notebook 実行寄り

数値はサンプルであり、自由に変更して試すことができます。

> Note: These scores are illustrative and represent typical usage tendencies, not language superiority.



In [ ]:
# generate_data

# Build a small, OSS-friendly dataset inside the notebook,
# then write it to CSV so the rest of the pipeline is reproducible.

data = [
    ("Ruby",   "DSL/Orchestration",     5),
    ("Ruby",   "Data Wrangling",         4),
    ("Ruby",   "Notebook Workflow",      2),
    ("Ruby",   "Visualization",          3),
    ("Ruby",   "Packaging/Deployment",   5),
    ("Python", "DSL/Orchestration",      3),
    ("Python", "Data Wrangling",         5),
    ("Python", "Notebook Workflow",      5),
    ("Python", "Visualization",          5),
    ("Python", "Packaging/Deployment",   4),
]

df = pd.DataFrame(data, columns=["language", "dimension", "score"])
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df.to_csv(csv_path, index=False)

print(f"generated: {csv_path}")
df.head(10)


# graph_output

レーダーチャートをファイルとして出力する。

In [ ]:
# graph_output

# Read CSV and plot a radar chart to a PNG file.

df = pd.read_csv(csv_path)

dims = split_or_none(dimension_order) or df["dimension"].unique().tolist()
langs = split_or_none(lang_order) or df["language"].unique().tolist()

pivot = df.pivot(index="dimension", columns="language", values="score")
pivot = pivot.reindex(index=dims, columns=langs).fillna(0)

N = len(dims)
angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
angles += angles[:1]  # close the loop

fig = plt.figure(figsize=(7, 7))
ax = plt.subplot(111, polar=True)

# Put first axis at the top and go clockwise
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

ax.set_ylim(0, max_score)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(dims)

# Y ticks (1..max_score)
ax.set_yticks(list(range(1, int(max_score) + 1)))
ax.set_yticklabels([str(i) for i in range(1, int(max_score) + 1)])

for lang in langs:
    values = pivot[lang].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, label=lang)
    ax.fill(angles, values, alpha=fill_alpha)

ax.set_title(title, pad=20)
ax.legend(loc="upper right", bbox_to_anchor=(1.25, 1.15))

plt.tight_layout()
plt.savefig(out_png, dpi=150)
print(f"saved: {out_png}")


# graph_view

生成したグラフを Notebook 上で可視化する。

※ このセルは `graph_output` により生成された画像を表示します。
そのため、単独では実行できません。


In [ ]:
# graph_view

# Optional: display the generated image in the notebook.
from IPython.display import Image, display

if show_plot:
    display(Image(filename=out_png))
else:
    print("show_plot is False (set it True to display the image).")
